<a href="https://colab.research.google.com/github/Speedbird45Bravo/Keras_projects_21/blob/main/Keras_4821_sand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()
df = df[(df.league=='Barclays Premier League') | (df.league=='English League Championship') | (df.league=='Major League Soccer')]
df['spi_margin'] = np.abs(df['spi1'] - df['spi2'])
df['margin'] = np.abs(df['score1'] - df['score2'])
df['totgoals'] = df['score1'] + df['score2']

res = []

for x in df['totgoals']:
  if x > 0:
    res.append("GOALS")
  else:
    res.append("SCORELESS")

df['goals'] = res

In [7]:
def load_model():
  model = Sequential()
  model.add(Dense(256, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(16, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

def to_cat(array):
  y = to_categorical(np.asarray(pd.factorize(array)[0]))
  return y

def to_df(array):
  df = pd.DataFrame(array).reset_index(drop=True)
  return df

In [8]:
model = load_model()

In [13]:
X = df[['score1','score2']].reset_index(drop=True)
y = df[['goals']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [14]:
y_train = to_cat(y_train['goals'])
y_train = to_df(y_train)
y_test = to_cat(y_test['goals'])
y_test = to_df(y_test)

In [15]:
model.fit(X_train, y_train, epochs=40, batch_size=128, verbose=1, validation_split=0.1)

Epoch 1/40
29/29 [==============================] - 0s 13ms/step - loss: 9.1854e-07 - accuracy: 1.0000 - val_loss: 7.0423e-07 - val_accuracy: 1.0000
Epoch 2/40
29/29 [==============================] - 0s 4ms/step - loss: 8.9633e-07 - accuracy: 1.0000 - val_loss: 6.9321e-07 - val_accuracy: 1.0000
Epoch 3/40
29/29 [==============================] - 0s 4ms/step - loss: 8.8412e-07 - accuracy: 1.0000 - val_loss: 6.7001e-07 - val_accuracy: 1.0000
Epoch 4/40
29/29 [==============================] - 0s 4ms/step - loss: 8.5723e-07 - accuracy: 1.0000 - val_loss: 6.5521e-07 - val_accuracy: 1.0000
Epoch 5/40
29/29 [==============================] - 0s 4ms/step - loss: 8.3730e-07 - accuracy: 1.0000 - val_loss: 6.4419e-07 - val_accuracy: 1.0000
Epoch 6/40
29/29 [==============================] - 0s 4ms/step - loss: 8.2436e-07 - accuracy: 1.0000 - val_loss: 6.3868e-07 - val_accuracy: 1.0000
Epoch 7/40
29/29 [==============================] - 0s 4ms/step - loss: 8.1286e-07 - accuracy: 1.0000 - val_los

In [16]:
predictions = model.predict(X_test).round(0)
pred2 = model.predict(X_train)

In [17]:
loss2, acc2 = model.evaluate(y_train, pred2)
loss, acc = model.evaluate(y_test, predictions)

33/33 [==============================] - 0s 1ms/step - loss: 1.1641 - accuracy: 0.9211


In [18]:
print("Train Accuracy: %.2f" % (acc2 * 100) + "%")
print("Test Accuracy: %.2f" % (acc * 100) + "%")

Train Accuracy: 93.44%
Test Accuracy: 92.11%
